# Spotify Analysis

First we install the necessary libraries needed for data visualization and wrangling. Additionally, the R wrapper, Spotifyr, which allows us to get insights from Spotify's web api is installed as well. 

In [33]:
library(tidyverse)
library(spotifyr)
library(repr)

Below, we access the web API using the Client ID and Client Secret provided by Spotify upon creating a Developer Account.

In [34]:
access_token <- get_spotify_access_token()

### A Comparison Of My Playlists From Year 1 and Year 2 Of Undergrad
I have attempted to capture each year of my undergraduate degree in a unique playlist. So far, I have one and a half years' worth of data. I will begin my analysis using a playlist I began creating in the fall of 2022, my first year of university. I am interested in learning the general atmosphere of the playlist and how my music preference has changed throughout my first and second years of university. 

> While reading my playlist data, I realized that Spotify has a limit that only allows 100 items to be read at a time. Therefore, to add all 566 items of my playlist here, I set the offsets parameter to increment by 100 until it reaches 500. Then I map each of these offsets to the data frames of 100 tracks, starting from each offset value. Finally I combine all the separate data frames into a single data frame. 

In [35]:
# Loading the playlist data frame into the "playlist_2022" variable

playlist_id <- "2cQLZcVSOLDfY03SY5DZrf"
offsets <- seq(0, 500, 100)

# Mapping offsets
playlist_tracks <- map(offsets, ~ get_playlist_tracks(playlist_id, limit = 100, offset = .x))

# Combining the results into a single data frame
playlist_2022 <- bind_rows(playlist_tracks) |> as_tibble()


In [36]:
# Getting all the column names of the data frame
colnames(playlist_2022)

[1] "added_at"                           "is_local"                          
 [3] "primary_color"                      "added_by.href"                     
 [5] "added_by.id"                        "added_by.type"                     
 [7] "added_by.uri"                       "added_by.external_urls.spotify"    
 [9] "track.artists"                      "track.available_markets"           
[11] "track.disc_number"                  "track.duration_ms"                 
[13] "track.episode"                      "track.explicit"                    
[15] "track.href"                         "track.id"                          
[17] "track.is_local"                     "track.name"                        
[19] "track.popularity"                   "track.preview_url"                 
[21] "track.track"                        "track.track_number"                
[23] "track.type"                         "track.uri"                         
[25] "track.album.album_type"             "track.album.artists"               
[27] "track.album.available_markets"      "track.album.href"                  
[29] "track.album.id"                     "track.album.images"                
[31] "track.album.name"                   "track.album.release_date"          
[33] "track.album.release_date_precision" "track.album.total_tracks"          
[35] "track.album.type"                   "track.album.uri"                   
[37] "track.album.external_urls.spotify"  "track.external_ids.isrc"           
[39] "track.external_urls.spotify"        "video_thumbnail.url"

Based on the above columns, some interesting columns to analyze include: 
- 'added_at' - The date and time the song was added (NA is used if playlist is too old)
- 'track.artists' - 
- 'track.duration_ms'
- 'track.explicit'
- 'track.name'
- 'track.popularity'
- 'track.album.album_type'
- 'track.album.name'
- 'track.album.release_date'

In [38]:
# Selecting only the identified columns above
shortened_playlist_2022 <- playlist_2022 |> select(added_at, track.artists, track.duration_ms, 
                                                  track.explicit, track.name, track.popularity, 
                                                  track.album.album_type, track.album.name, 
                                                  track.album.release_date)
head(shortened_playlist_2022)

added_at,track.artists,track.duration_ms,track.explicit,track.name,track.popularity,track.album.album_type,track.album.name,track.album.release_date
<chr>,<list>,<int>,<lgl>,<chr>,<int>,<chr>,<chr>,<chr>
2022-08-06T00:01:24Z,"https://api.spotify.com/v1/artists/5GGJosGMs08YEmKTZJe1fL, 5GGJosGMs08YEmKTZJe1fL , Suki Waterhouse , artist , spotify:artist:5GGJosGMs08YEmKTZJe1fL , https://open.spotify.com/artist/5GGJosGMs08YEmKTZJe1fL",214800,FALSE,Good Looking,1,single,Good Looking,2017-10-20
2022-08-06T00:04:37Z,"https://api.spotify.com/v1/artists/5ZsFI1h6hIdQRw2ti0hz81, 5ZsFI1h6hIdQRw2ti0hz81 , ZAYN , artist , spotify:artist:5ZsFI1h6hIdQRw2ti0hz81 , https://open.spotify.com/artist/5ZsFI1h6hIdQRw2ti0hz81",202213,FALSE,fOoL fOr YoU,65,album,Mind Of Mine (Deluxe Edition),2016-03-25
2022-08-06T00:17:48Z,"https://api.spotify.com/v1/artists/2kxP07DLgs4xlWz8YHlvfh, 2kxP07DLgs4xlWz8YHlvfh , NIKI , artist , spotify:artist:2kxP07DLgs4xlWz8YHlvfh , https://open.spotify.com/artist/2kxP07DLgs4xlWz8YHlvfh",215686,FALSE,Every Summertime,77,single,Every Summertime,2021-08-10
2022-08-06T00:18:07Z,"https://api.spotify.com/v1/artists/6nxWCVXbOlEVRexSbLsTer, https://api.spotify.com/v1/artists/6xHUXzrfhFgnIv86EBR3Ml, 6nxWCVXbOlEVRexSbLsTer , 6xHUXzrfhFgnIv86EBR3Ml , Flume , kai , artist , artist , spotify:artist:6nxWCVXbOlEVRexSbLsTer , spotify:artist:6xHUXzrfhFgnIv86EBR3Ml , https://open.spotify.com/artist/6nxWCVXbOlEVRexSbLsTer , https://open.spotify.com/artist/6xHUXzrfhFgnIv86EBR3Ml",233337,TRUE,Never Be Like You,0,album,Skin,2016-05-27
2022-08-06T00:26:55Z,"https://api.spotify.com/v1/artists/3pc0bOVB5whxmD50W79wwO, 3pc0bOVB5whxmD50W79wwO , keshi , artist , spotify:artist:3pc0bOVB5whxmD50W79wwO , https://open.spotify.com/artist/3pc0bOVB5whxmD50W79wwO",164662,TRUE,SOMEBODY,68,album,GABRIEL,2022-03-25
2022-08-10T04:30:57Z,"https://api.spotify.com/v1/artists/2joIhhX3Feq47H4QXVDOr3, 2joIhhX3Feq47H4QXVDOr3 , RINI , artist , spotify:artist:2joIhhX3Feq47H4QXVDOr3 , https://open.spotify.com/artist/2joIhhX3Feq47H4QXVDOr3",257000,FALSE,My Favourite Clothes,66,single,My Favourite Clothes,2018-01-06
